# Introduction
In this assignment, you are going to train a neural network to **predict a likely language of origin for a last name**.


- Learn to preprocess raw text so it can be fed into an RNN and LSTM;
- Make use of relevant libraries in Pytorch;
- Train, evaluate, and generate predictions for unseen data.

Disclaimer: we’re using this dataset for academic purposes only. The model you train should not be used to draw any real-world conclusions about people’s names.



# Library imports

Before starting, make sure you have all these libraries.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# TODO: cd into the correct drive folder! Example:
%cd drive/MyDrive/hw3

/content/drive/MyDrive/GSI/GNN_RNN/hw3_sp


In [ ]:
root_folder = ""
import os
import sys
sys.path.append(root_folder)
import torch as th
from torch import nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
device = th.device("cuda" if th.cuda.is_available() else "cpu")

In [ ]:
%load_ext autoreload
%autoreload 2

# Part A. Creating the Network
The problem is predicting the $$ P ( class | c_1, c_2, c_3, ..., c_i), \forall i \in name $$

where c is each character in a name. The advantage of building this basic character-level NLP model is that you can see how preprocessing works at a low level. A character-level network reads words as a series of characters - outputting a prediction and “hidden state” at each step, feeding its previous hidden state into each next step. We take the final prediction to be the output, i.e. which class the word belongs to.

All models in PyTorch inherit from the subclass nn.Module. Here, you will intialize a custom model called RecurrentClassifier to run [nn.RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) or [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html), among other nn functions you can find in the Pytorch docs, and define its forward function. Fill in the TODO's below. **Note: set batch_first in the arguments to the above to True.**

In [ ]:
class RecurrentClassifier(nn.Module):
    def __init__(self, vocab_size, rnn_size, n_categories, model_type, num_layers=1, dropout=0):
        super().__init__()
        self.model_type = model_type
        # An embedding layer that will map our vocab into a vector of length rnn_size. 
        self.embedding = # TODO

        # Create an LSTM layer of rnn_size size. Use any features you wish.
        # We will be using batch_first convention
        if model_type == 'lstm':
            self.lstm = # TODO
        elif model_type == 'rnn':
            self.rnn = # TODO
        # RNN/LSTM do not add dropout to the last hidden output, 
        # uncomment the line below if you wish.
        # self.dropout = nn.Dropout(p=dropout)

        # Final linear layer that projects the outputs of the RNN/LSTM into
        # logits of size n_categories
        self.output = # TODO

    def forward(self,x):
        '''
        Run input data x through each relevant layer depending on self.model_type,
        and output the final logits.
        '''
        # TODO


# Part B: Preprocessing the dataset
The [dataset](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html) contains a few thousand last names from 18 languages of origin. Included in the data/names directory are 18 text files named as "[Language].txt". Each file contains a bunch of names, one name per line, mostly romanized (but we still need to convert from Unicode to ASCII).

We'll end up with a dictionary of lists of names per language, {language: [names ...]}. 

Fill in the TODO's in the following cells.


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print(category_lines['Italian'][:5])

['data/names/French.txt', 'data/names/Polish.txt', 'data/names/Scottish.txt', 'data/names/Japanese.txt', 'data/names/German.txt', 'data/names/English.txt', 'data/names/Irish.txt', 'data/names/Greek.txt', 'data/names/Russian.txt', 'data/names/Dutch.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Arabic.txt', 'data/names/Spanish.txt', 'data/names/Portuguese.txt', 'data/names/Korean.txt', 'data/names/Italian.txt', 'data/names/Vietnamese.txt']
Slusarski


In [ ]:
def letterToIndex(letter):
    '''
    Find letter index from all_letters, e.g. "a" returns 0
    '''
    # TODO

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [ ]:
# For labels, we must have numbers instead of a string. These dictionaries convert
# between these two ways of representing the labels.
num_to_cat = dict(enumerate(category_lines.keys()))
cat_to_num = dict((v,k) for k,v in num_to_cat.items())
pad = 57 # this is the next available character 
vocab_size = 58 # number of characters used in total

def build_data(category_lines):
  '''
  category_lines: a dictionary of lists of names per language, {language: [names ...]}. 

  We want to translate our dictionary into a dataset that has one entry per name. 
  Each datapoint is a 3-tuple consisting of: 
  - x: a length-19 array with each character in the name as an element,
   padded with zeros at the end if the name is less than 19 characters.
  - y: the numerical representation of the language the name corresponds to.
  - index: the index where the padding begins
  '''
  data = []
  for cat in category_lines:
    for name in category_lines[cat]:
      token = np.ones(19) * pad
      numerized = np.array([letterToIndex(l) for l in name])
      n = len(numerized)
      token[:n] = numerized
      data.append((token, cat_to_num[cat], n-1))
  return data

NameError: ignored

In [ ]:
data = build_data(category_lines)
# TODO: shuffle the dataset, then do train test split with 80% as train set.
train_data = # TODO
test_data = # TODO

# Part C: Training the models

Fill in the TODO's below. 

Training will be faster if you use the Colab GPU. If it's not already enabled, do so with Runtime -> Change runtime type.

Look at the training loss curves of RNN vs. LSTM. Which one does better? 

In [ ]:
def build_batch(dataset, indices):
    '''
    Helper function for creating a batch during training. Builds a batch 
    of source and target elements from the dataset. See the next cell for 
    when and how it's used. 
    
    Arguments:
        dataset: List[db_element] -- A list of datapoints.
        Again, each datapoint is a 3-tuple consisting of: 
          - x: a length-19 array with each character in the name as an element,
          padded with zeros at the end if the name is less than 19 characters.
          - y: the numerical representation of the language the name corresponds to.
          - index: the index where the padding begins

        indices: List[int] -- A list of indices of the dataset to sample
    Returns:
        batch_input: np.array[np.array[int]] -- List of tensorized names
        batch_target: np.array[int] -- List of numerical categories
        batch_indices: np.array[int] -- List of starting indices of padding
    '''
    # Recover what the entries for the batch are
    batch = [dataset[i] for i in indices]
    # TODO
    return batch_input, batch_target, batch_indices

In [ ]:
criterion = # TODO: use Pytorch's cross-entropy loss.
def loss_fn(pred, target, index):
    '''
    Arguments:
      pred: tensor of shape (batch_size, 19, vocab_size), representing predictions
      for all 19 timesteps, one character per timestep
      target: labels of length batch_size
      index: indices of length batch_size
    Returns:
      Loss averaged acrosss the batch. The criterion already handles the averaging.
    '''
    # we just want to compute loss based on the prediction's last token
    pred_valid = pred[np.arange(len(pred)),index,:] 
    loss_tensor = # TODO: use criterion
    return loss_tensor

# The build_batch function outputs numpy, but our model is built in pytorch,
# so we need to convert numpy to pytorch with the correct types. 
batch_to_torch = lambda b_in,b_target,b_mask: (th.tensor(b_in).long(),
                                               th.tensor(b_target).long(), 
                                               th.tensor(b_mask).long())

# Model parameters. You can tune this for better performance. 
hidden_size = 256
num_layers = 2
dropout = .3
optimizer_class = optim.Adam
lr = 1e-3
epochs = 10
batch_size = 128

device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)
list_to_device = lambda th_obj: [tensor.to(device) for tensor in th_obj]

cuda


In [ ]:
# invoke the model class you defined earlier. 
lstm_model = # TODO
lstm_optimizer = optimizer_class(lstm_model.parameters(), lr=lr)
rnn_model = # TODO
rnn_optimizer = optimizer_class(rnn_model.parameters(), lr=lr)

In [ ]:
def train(model, optimizer, epochs, model_id='test_1'):
  '''
  Train and save a model.

  Make sure to save your latest model with a UNIQUE model name if you need to
  access it later! 
  '''
  model.train()
  losses = []
  accuracies = []
  from tqdm import tqdm
  for epoch in range(epochs):
      indices = np.random.permutation(range(len(train_data)))
      t = tqdm(range(0,(len(train_data)//batch_size)+1))
      for i in t:
          batch = # TODO: make a batch from training data, depending on iteration i and batch_size
          (batch_input, batch_target, batch_indices) = batch_to_torch(*batch)
          (batch_input, batch_target, batch_indices) = list_to_device((batch_input, batch_target, batch_indices))
          model.to(device)
          
          prediction = # TODO
          loss = # TODO
          losses.append(loss.item())
          pred_valid = prediction[np.arange(len(prediction)),batch_indices,:]
          accuracy = th.mean(th.eq(pred_valid.argmax(dim=1,keepdim=False),batch_target).float())
          accuracies.append(accuracy.item())

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          if i % 10 == 0:
              t.set_description(f"Epoch: {epoch} Iteration: {i} Loss: {np.mean(losses[-10:])} Accuracy: {np.mean(accuracies[-10:])}")
      os.makedirs(root_folder+'models/part1',exist_ok=True)        
      save_dict = dict(
          kwargs = dict(
              vocab_size=vocab_size,
              rnn_size=hidden_size,
              n_categories=n_categories,
              num_layers=num_layers,
              dropout=dropout
          ),
          model_state_dict = model.state_dict(),
          notes = "",
          optimizer_class = optimizer_class,
          lr = lr,
          epochs = epochs,
          batch_size = batch_size,
      )
      th.save(save_dict,root_folder+f'models/part1/model_{model_id}.pt')
  return losses

In [ ]:
lstm_losses = train(lstm_model, lstm_optimizer, epochs, 'test_lstm')

Epoch: 0 Iteration: 120 Loss: 1.2119470715522767 Accuracy: 0.65859375: 100%|██████████| 126/126 [00:02<00:00, 47.04it/s]
Epoch: 1 Iteration: 120 Loss: 0.96001495718956 Accuracy: 0.7296875: 100%|██████████| 126/126 [00:02<00:00, 48.91it/s]
Epoch: 2 Iteration: 120 Loss: 0.7745696604251862 Accuracy: 0.77578125: 100%|██████████| 126/126 [00:02<00:00, 48.42it/s]
Epoch: 3 Iteration: 120 Loss: 0.7311235904693604 Accuracy: 0.78125: 100%|██████████| 126/126 [00:02<00:00, 48.36it/s]
Epoch: 4 Iteration: 120 Loss: 0.6932623147964477 Accuracy: 0.79453125: 100%|██████████| 126/126 [00:02<00:00, 47.93it/s]
Epoch: 5 Iteration: 120 Loss: 0.5932267665863037 Accuracy: 0.8203125: 100%|██████████| 126/126 [00:02<00:00, 47.81it/s]
Epoch: 6 Iteration: 120 Loss: 0.5783548861742019 Accuracy: 0.82265625: 100%|██████████| 126/126 [00:02<00:00, 43.84it/s]
Epoch: 7 Iteration: 120 Loss: 0.5618159711360932 Accuracy: 0.83515625: 100%|██████████| 126/126 [00:02<00:00, 47.00it/s]
Epoch: 8 Iteration: 120 Loss: 0.5211859

In [ ]:
rnn_losses = train(rnn_model, rnn_optimizer, epochs, 'test_rnn')

Epoch: 0 Iteration: 120 Loss: 1.1844713270664216 Accuracy: 0.65390625: 100%|██████████| 126/126 [00:01<00:00, 113.70it/s]
Epoch: 1 Iteration: 120 Loss: 0.8666493415832519 Accuracy: 0.740625: 100%|██████████| 126/126 [00:01<00:00, 123.29it/s]
Epoch: 2 Iteration: 120 Loss: 0.8606041729450226 Accuracy: 0.7578125: 100%|██████████| 126/126 [00:01<00:00, 123.85it/s]
Epoch: 3 Iteration: 120 Loss: 0.757456761598587 Accuracy: 0.78671875: 100%|██████████| 126/126 [00:01<00:00, 123.25it/s]
Epoch: 4 Iteration: 120 Loss: 0.693710321187973 Accuracy: 0.8: 100%|██████████| 126/126 [00:01<00:00, 123.26it/s]
Epoch: 5 Iteration: 120 Loss: 0.6759132534265518 Accuracy: 0.790625: 100%|██████████| 126/126 [00:01<00:00, 123.63it/s]
Epoch: 6 Iteration: 120 Loss: 0.7112273216247559 Accuracy: 0.79296875: 100%|██████████| 126/126 [00:01<00:00, 123.70it/s]
Epoch: 7 Iteration: 120 Loss: 0.5883178889751435 Accuracy: 0.8265625: 100%|██████████| 126/126 [00:01<00:00, 122.81it/s]
Epoch: 8 Iteration: 120 Loss: 0.5593171

In [ ]:
plt.title("RNN vs. LSTM train losses over earlier timesteps")
plt.plot(lstm_losses,label="LSTM")
plt.plot(rnn_losses, label="RNN")
plt.xlim([0,800])
plt.legend()

# Part D: Using the LSTM model

## (1) Evaluation loss

Load the model you trained, and obtain a test loss. Your objective is to reach a **LSTM validation loss <= 0.6**
You should not need more than 10 epochs to attain the threshold. More passes over the data can however give you a better model. Other options include changing your optimizers, tuning your learning_rate, and using the last dropout layer.
Fill in the TODO's.

In [ ]:
model_id = "test1" # Make sure this id matches the id you assigned the LSTM model
save_dict = th.load(root_folder+'models/part1/'+f"model_{model_id}.pt",)
model = RecurrentClassifier(**save_dict['kwargs'],model_type='lstm')
model.load_state_dict(save_dict['model_state_dict'])
model.eval()

LanguageModel(
  (embedding): Embedding(58, 512)
  (lstm): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.5)
  (output): Linear(in_features=512, out_features=58, bias=True)
)

In [ ]:
# Make sure your validation loss is below 0.6 and that you didn't train using
# the test set.
batch = # TODO: put entire test set into a batch
(batch_input, batch_target, batch_indices) = batch_to_torch(*batch)
prediction = # TODO
loss = loss_fn(prediction, batch_target, batch_indices)
print("Evaluation set loss:", loss.item())

IndexError: ignored

In [ ]:
model.cpu()

## (2) Try some other names!
This time, obtain the output_logits from our model instead of the loss to predict on a name that you specify. Fill in the TODO's.

In [ ]:
def classify_name(name, model):
    '''
    Numerize the name and return the most likely number representation of the 
    predicted class.
    '''
    token = np.ones(19) * pad
    numerized = np.array([letterToIndex(l) for l in name])
    n = len(numerized)
    token[:n] = numerized
    logits = # TODO
    logits_np = logits.detach().cpu().numpy()
    return # TODO

In [ ]:
model.eval()
names= ['Sahai']
for name in names:
    print("===================")
    print("Predicting origin language for name: "+ name)
    c = classify_name(name, model)
    print(num_to_cat[c])

Classifying nationality for name: Sahai
Arabic


## Woooo! All done. 
